# Assignment 4

Dependencies


In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz
!tar xf spark-3.2.0-bin-hadoop3.2.tgz
!pip install -q findspark

Environment variables

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "spark-3.2.0-bin-hadoop3.2"

Initializing pyspark

In [3]:
import findspark
findspark.init()

Check pyspark functionality

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=pyspark-shell>

**Mounting Google drive**

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [7]:
import re
from pyspark.sql import functions as F

#### Reading the reviews into an rdd

In [8]:
rdd = sc.wholeTextFiles("/content/drive/MyDrive/H516/Assignment 4/moviereviews/*.txt")

In [9]:
rdd.take(1)

[('file:/content/drive/MyDrive/H516/Assignment 4/moviereviews/cv000_29416.txt',
  'plot : two teen couples go to a church party , drink and then drive . \nthey get into an accident . \none of the guys dies , but his girlfriend continues to see him in her life , and has nightmares . \nwhat\'s the deal ? \nwatch the movie and " sorta " find out . . . \ncritique : a mind-fuck movie for the teen generation that touches on a very cool idea , but presents it in a very bad package . \nwhich is what makes this review an even harder one to write , since i generally applaud films which attempt to break the mold , mess with your head and such ( lost highway & memento ) , but there are good and bad ways of making all types of films , and these folks just didn\'t snag this one correctly . \nthey seem to have taken this pretty neat concept , but executed it terribly . \nso what are the problems with the movie ? \nwell , its main problem is that it\'s simply too jumbled . \nit starts off " normal " b

In [10]:
rdd_words = rdd.map(lambda x:  (x[0][59:],x[1].lower()))

In [11]:
rdd_words.take(1)

[('cv000_29416.txt',
  'plot : two teen couples go to a church party , drink and then drive . \nthey get into an accident . \none of the guys dies , but his girlfriend continues to see him in her life , and has nightmares . \nwhat\'s the deal ? \nwatch the movie and " sorta " find out . . . \ncritique : a mind-fuck movie for the teen generation that touches on a very cool idea , but presents it in a very bad package . \nwhich is what makes this review an even harder one to write , since i generally applaud films which attempt to break the mold , mess with your head and such ( lost highway & memento ) , but there are good and bad ways of making all types of films , and these folks just didn\'t snag this one correctly . \nthey seem to have taken this pretty neat concept , but executed it terribly . \nso what are the problems with the movie ? \nwell , its main problem is that it\'s simply too jumbled . \nit starts off " normal " but then downshifts into this " fantasy " world in which you

Creating lists of the positive and negative words

In [12]:
with open('/content/drive/MyDrive/H516/Assignment 4/pos.txt') as f:
    posWords = f.readlines()
i=0
for word in posWords:
    posWords[i] = word.strip()
    i+=1

In [13]:
with open('/content/drive/MyDrive/H516/Assignment 4/neg.txt') as f:
    negWords = f.readlines()
i=0
for word in negWords:
    negWords[i] = word.strip()
    i+=1

Removing "*" from negative words for regexp

In [14]:
negWords = [x for x in negWords if "*" not in x]

Converting rdd to dataframe

In [15]:
df = rdd_words.toDF()

In [20]:
df = df.withColumnRenamed("_1","FILE").withColumnRenamed("_2","REVIEW")
df.show(5)

+---------------+--------------------+--------------------+--------------------+
|           FILE|              REVIEW|                 POS|                 NEG|
+---------------+--------------------+--------------------+--------------------+
|cv000_29416.txt|plot : two teen c...|[cool, applaud, g...|[plot, dies, fuck...|
|cv000_29590.txt|films adapted fro...|[success, like, l...|[hell, odd, hell,...|
|cv001_18431.txt|every now and the...|[darling, popular...|[suspect, critica...|
|cv001_19502.txt|the happy bastard...|[happy, like, fla...|[bastard, damn, b...|
|cv002_15918.txt|you've got mail w...|[works, better, s...|[twists, terribly...|
+---------------+--------------------+--------------------+--------------------+
only showing top 5 rows



Converting positive and negative lists to regexp form

In [21]:
re_pos = '(' + '|'.join([fr'\\b{k}\\b' for k in posWords]) + ')'
re_neg = '(' + '|'.join([fr'\\b{k}\\b' for k in negWords]) + ')'

Finding all positive and negaitve words from the reviews

In [22]:
df = df.withColumn('POS', F.expr(f"regexp_extract_all(REVIEW, '{re_pos}', 1)")).withColumn('NEG', F.expr(f"regexp_extract_all(REVIEW, '{re_neg}', 1)"))
df.show(5)

+---------------+--------------------+--------------------+--------------------+
|           FILE|              REVIEW|                 POS|                 NEG|
+---------------+--------------------+--------------------+--------------------+
|cv000_29416.txt|plot : two teen c...|[cool, applaud, g...|[plot, dies, fuck...|
|cv000_29590.txt|films adapted fro...|[success, like, l...|[hell, odd, hell,...|
|cv001_18431.txt|every now and the...|[darling, popular...|[suspect, critica...|
|cv001_19502.txt|the happy bastard...|[happy, like, fla...|[bastard, damn, b...|
|cv002_15918.txt|you've got mail w...|[works, better, s...|[twists, terribly...|
+---------------+--------------------+--------------------+--------------------+
only showing top 5 rows



Getting the total counts of positive and negative words from each review

In [23]:
df = df.withColumn('posCount', F.size(F.col('POS'))).withColumn('negCount', F.size(F.col('NEG')))
df.show(5)

+---------------+--------------------+--------------------+--------------------+--------+--------+
|           FILE|              REVIEW|                 POS|                 NEG|posCount|negCount|
+---------------+--------------------+--------------------+--------------------+--------+--------+
|cv000_29416.txt|plot : two teen c...|[cool, applaud, g...|[plot, dies, fuck...|      26|      26|
|cv000_29590.txt|films adapted fro...|[success, like, l...|[hell, odd, hell,...|      24|      39|
|cv001_18431.txt|every now and the...|[darling, popular...|[suspect, critica...|      27|      24|
|cv001_19502.txt|the happy bastard...|[happy, like, fla...|[bastard, damn, b...|      10|       6|
|cv002_15918.txt|you've got mail w...|[works, better, s...|[twists, terribly...|      30|      15|
+---------------+--------------------+--------------------+--------------------+--------+--------+
only showing top 5 rows



Deciding sentiment based on comparing the counts of positive and negative words (displayed final output)

In [24]:
df = df.withColumn('SENTIMENT', F.when((F.col("posCount") > F.col("negCount")), "Positive") \
    .otherwise(F.when((F.col("posCount") < F.col("negCount")), "Negative") \
    .otherwise("Undecided")))
df.show()

+---------------+--------------------+--------------------+--------------------+--------+--------+---------+
|           FILE|              REVIEW|                 POS|                 NEG|posCount|negCount|SENTIMENT|
+---------------+--------------------+--------------------+--------------------+--------+--------+---------+
|cv000_29416.txt|plot : two teen c...|[cool, applaud, g...|[plot, dies, fuck...|      26|      26|Undecided|
|cv000_29590.txt|films adapted fro...|[success, like, l...|[hell, odd, hell,...|      24|      39| Negative|
|cv001_18431.txt|every now and the...|[darling, popular...|[suspect, critica...|      27|      24| Positive|
|cv001_19502.txt|the happy bastard...|[happy, like, fla...|[bastard, damn, b...|      10|       6| Positive|
|cv002_15918.txt|you've got mail w...|[works, better, s...|[twists, terribly...|      30|      15| Positive|
|cv002_17424.txt|it is movies like...|[like, thankful, ...|[jaded, wrong, st...|      24|      18| Positive|
|cv003_11664.txt| "